# Prompt Engineering: Use OpenAI to Analyze Twitter Data 
This is a simple tutorial teaching prompt engineering basics and analyzing Twitter data with OpenAI large language models (LLM).
Please purchase an [OpenAI API](https://openai.com/index/openai-api/) and store it in a safe place. This tutorial uses [AWS Secretes Manager](https://aws.amazon.com/secrets-manager/) to store the API keys.  

## Large Language Model Basics
LLM repeatable predicts the next world using supervised learning. To predict the following sentence: 

`Learning data science in the cloud with AI`

A model needs to learn to predict the following steps:

|Input|Output|
|:---|---|
|Learning data science |in |
|Learning data science in |the | 
|Learning data science in the |cloud |
|Learning data science in the cloud |with |
|Learning data science in the cloud with |AI|

To train an LLM model:
1. Training a base LLM model on a large amount of training data to predict the next word 
2. Fine-tune on examples where outputs follow instructions in the input 
3. Human rates quality of different LLM outputs 
4. Tune LLM to generate outputs with higher rates using RLHF (Reinforcement learning from human feedback)

## Set up OpenAI Models

Load the API keys with AWS Secrets Manage Function 

In [1]:
import boto3
from botocore.exceptions import ClientError
import json

def get_secret(secret_name):
    region_name = "us-east-1"

    # Create a Secrets Manager client
    session = boto3.session.Session()
    client = session.client(
        service_name='secretsmanager',
        region_name=region_name
    )

    try:
        get_secret_value_response = client.get_secret_value(
            SecretId=secret_name
        )
    except ClientError as e:
        raise e

    secret = get_secret_value_response['SecretString']
    
    return json.loads(secret)

## Install Python libraries.

- pymongo: manage the MongoDB database
- openai: call the OpenAI APIs.

In [2]:
pip install openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 647.0/647.0 kB 37.2 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install pymongo

Note: you may need to restart the kernel to use updated packages.


Load the OpenAI API key and define a `openai_help` function.

In [4]:
from openai import OpenAI

openai_api_key  = get_secret('openai')['api_key']
client = OpenAI(api_key=openai_api_key)
model = 'gpt-4o'
temperature = 0

def openai_help(messages, model=model, temperature =temperature ):
    messages = messages
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature

    )
    return response.choices[0].message.content

Temperature: 
- Low temperature: always choose the most likely response, reliable, predictable responses  
- High temperature: diverse responses, more creative responses

Tokens and Models: 
- LLM predicts tokens, which are commonly occurring sequences of characters. 
- One token is about four characters in English, and 100 tokens are roughly 75 words. Check [token estimate](https://platform.openai.com/tokenizer).
- Different models can process various amounts of tokens at different performance levels and costs. Check [OpenAI models](https://platform.openai.com/docs/models) for more details.

Roles:
- system: specify the overall tone or behavior of the assistant 
- user: instruction given to the LLM
- assistant: LLM responded content, we also can provide content in few-shot promoting or histories of conversations


A simple example using [gtp-4o](https://platform.openai.com/docs/models/gpt-4o) and temperature 0.

In [5]:
messages = [{"role": "user", "content": "What is the capital of USA"}]

print(openai_help(messages))

The capital of the United States is Washington, D.C.


Add a system message asking LLM to act as a high school teacher with different temperatures.

In [6]:
messages = [
    {"role": "system", "content": "use tone as a high school teacher"},
    {"role": "user", "content": "What is the capital of USA"}
    ]

print(openai_help(messages, temperature = 0.8))

The capital of the United States is Washington, D.C. Make sure you remember this, as it's a commonly asked question in geography!


Add assistant messages to teach LLM what `##` is.

In [7]:
messages = [
    {"role": "user", "content": "What is 1##1"},
    {"role": "assistant", "content": "it is 11"},
    {"role": "user", "content": "What is 2##2"},
    {"role": "assistant", "content": "it is 22"},
    {"role": "user", "content": "What is 3##3"},
    ]
print(openai_help(messages))

The expression "3##3" does not have a standard mathematical interpretation. If you are referring to a specific operation or notation, please provide more context. If you are following a pattern similar to previous examples where "1##1" was interpreted as "11" and "2##2" as "22," then "3##3" would be "33."


## Prompt Engineering Principles 
- Use delimiters to separate different parts of a prompt to provide clear instructions and prevent prompt injections.
- Structure outputs in JSON documents or other formats to use the outputs in subsequent steps 
- Few-shot promoting: provide successful examples of a task and then ask the model to perform a similar task. 
- Chain of thought reasoning: request a series of reasoning steps in prompts to help the model achieve correct answers
- Chain of prompts: split a task into multiple prompts where each prompt can focus on a sub-task at a time and take different actions at different stages. It saves tokens, is easier to test, can involve human input, or use external tools.
- Interactive process 
  1. Try something first 
  2. Analyses the result, identify errors, and redefine the prompt 
  3. Test the prompts with different datasets 


An example using delimiters, structured output and few-shot promoting:

In [8]:
delimiter = '###'
sentence1 = 'I love cat.'
sentence2 = 'I love dog.'
messages = [
    {"role": "system", "content": f"""analyze the sentiment in a sentence delimitered by {delimiter},
                                     return the result as a JSON document"""},
    {"role": "user", "content": f"{delimiter}{sentence1}{delimiter}"},
    {"role": "assistant", "content": "{sentiment:positive}"},
    {"role": "user", "content": f"{delimiter}{sentence2}{delimiter}"}
    ]

print(openai_help(messages))

{ "sentiment": "positive" }


## Analyze Twitter data

### Connect to the MongoDB cluster

In [9]:
import pymongo
from pymongo import MongoClient
mongodb_connect = get_secret('mongodb')['connection_string']

mongo_client = MongoClient(mongodb_connect)
db = mongo_client.demo # use or create a database named demo
tweet_collection = db.tweet_collection #use or create a collection named tweet_collection
tweet_collection.create_index([("tweet.id", pymongo.ASCENDING)],unique = True) # make sure the collected tweets are unique

'tweet.id_1'

### Extract Tweets

In [10]:
filter={

    
}
project={
    'tweet.text': 1, 
    'tweet.id': 1
}
#rename the client to mongo_client
result = mongo_client['demo']['tweet_collection'].find(
  filter=filter,
  projection=project
)

In [11]:
tweet_data = []
for tweet in result:
    tweet_data.append(tweet['tweet']['text'])

In [12]:
print('Number of tweets: ',len(tweet_data))

Number of tweets:  93


### Summarization 
- Analyze election tweets with delimiters 
- Change the size of the summarization 
- Summarize tweets and focus on different perspectives. 

In [13]:
messages = [
    {"role": "system", "content": f"""provide a brief summary of the tweets delimited by {delimiter}"""},
    {"role": "user", "content": f"{delimiter}{tweet_data}{delimiter}"},
    ]

print(openai_help(messages))

The tweets primarily discuss various aspects and applications of generative AI. They highlight its integration in fields like IoT solutions, mobile and web development, and its role in enhancing weather forecasting and clinical data analysis. There are mentions of events and collaborations, such as Dubai AI Week 2025 and partnerships with AWS. Some tweets express skepticism or criticism of generative AI, particularly in art and journalism, while others emphasize its transformative potential in HR and marketing. Additionally, there are discussions about the ethical implications and the need for regulation in the use of generative AI.


In [14]:
messages = [
    {"role": "system", "content": f"""provide a brief summary of the tweets delimited by {delimiter},
                                    limit the summary to 20 words"""},
    {"role": "user", "content": f"{delimiter}{tweet_data}{delimiter}"},
    ]

print(openai_help(messages))

The tweets discuss generative AI's impact on various fields, including art, technology, and business, highlighting both opportunities and controversies.


In [15]:
messages = [
    {"role": "system", "content": f"""provide a brief summary of the tweets delimited by {delimiter},
                                    focus on how people discuss AI,
                                    limit the summary to 50 words"""},
    {"role": "user", "content": f"{delimiter}{tweet_data}{delimiter}"},
    ]

print(openai_help(messages))

People discuss AI with a focus on generative AI's impact across various fields, including art, music, and business. Opinions are mixed, with some praising its efficiency and potential, while others criticize its ethical implications and lack of creativity. Calls for regulation and ethical considerations are prevalent.


### Moderation 
- Iterate each tweet and use the [moeration endpoint](https://platform.openai.com/docs/api-reference/moderations) to identify flagged tweets
- Print flagged tweets


In [16]:
def flag_help(tweet):
    response = client.moderations.create(
        model="omni-moderation-latest",
        input=tweet)

    if response.results[0].flagged:
        print('===')
        cat_dict = response.results[0].categories.to_dict()
        for cat in cat_dict.keys():
            if cat_dict.get(cat):
                print (cat)
                print(tweet)

In [18]:
for tweet in tweet_data:
    flag_help(tweet)

===
harassment
@nic__carter @d_fins I think you misinterpret what people say. Ai is fine, generative Ai “art” that steals others work to mold it to what you want? That’s retarded lmao. You use absolutely no skill and your “art” (it’s not art cause it has no emotion) is quite literally jsut not art by definition
===
harassment
@nic__carter I'm more right leaning and fuck you and anyone else who supports the use of generative AI.
===
harassment
RT @clevelandy_: Fuck everyone who uses generative ai to create images for their fanfiction. I am better than you.
===
harassment
RT @clevelandy_: Fuck everyone who uses generative ai to create images for their fanfiction. I am better than you.
===
harassment
what's sad about ai artists is their clear lack of imagination. I've seen people make use of generative ai to produce really breathtaking art, but these people are the few. The larger majority of ai artists will just bastardize someone else art just like this below. https://t.co/laeHuR2U90
==

### Transforming
- Translating to a different language 
- Transform tones, such as formal vs. informal.  


In [19]:
for tweet in tweet_data:
    messages = [
        {"role": "system", "content": f"""translate the tweets delimited by {delimiter} into Chinese"""},
        {"role": "user", "content": f"{delimiter}{tweet}{delimiter} "}]

    print(openai_help(messages).strip(delimiter))

我们专注于： • 物联网解决方案 • 生成式AI集成 • 移动应用开发 • 网站开发 • 大学项目和MVP原型 快速、可靠且经济实惠！ 让我们一起打造一些很棒的东西。 了解更多：https://t.co/DXZqtPhy42 #与我们一起创新
RT @jradoff: 人工智能直播——主持人Jon Radoff（Beamable）欢迎Scenario的Emmanuel de Maistre，共同探讨创作者如何……
转发 @jradoff: 人工智能直播——主持人 Jon Radoff（Beamable）欢迎 Scenario 的 Emmanuel de Maistre 探讨创作者如何……
RT @DDNintelligence: DDN在Gartner最新的必读报告中被提及：
“CTO的生成式AI技术景观指南” 👇

为什么这…
RT @jradoff: 人工智能直播——主持人Jon Radoff（Beamable）邀请Scenario的Emmanuel de Maistre探讨创作者如何……
转发 @mdancho84: “生成式AI数据科学家”的崛起

这就是正在发生的事情：https://t.co/SBtpyKvJWY
大家好，我亲爱的天气爱好者和科技迷们！🌤️💻 这里是AisanDoll，今天我们要探讨一个和我的推特动态一样难以预测的话题——天气预报！但别担心，我有一个秘密武器：生成式AI。没错，就是那个
完成了第5讲！✨刚刚实现了我的第一个RAG系统！学习如何将文档检索与生成式AI结合。很高兴看到这如何减少幻觉并改善上下文处理！#GenAI #RAG #机器学习 #chaiaurcode
@piyushgarg_dev

@Hiteshdotcom https://t.co/NjuEv72emK
转发 @jradoff: 人工智能直播——主持人 Jon Radoff（Beamable）欢迎 Scenario 的 Emmanuel de Maistre 探讨创作者如何……
我还注意到你们持续未能解释为何未能遵循基本的新闻标准，使用生成式AI制作图像，而不是使用现有的库存图像等问题。
@nic__carter @d_fins 我认为你误解了人们所说的话。人工智能没问题，但生成式人工智能“艺术”窃取他人的作品来塑造成你想要的样子？那太愚蠢了，哈哈。你完全不需要任

In [20]:
for tweet in tweet_data:
    messages = [
        {"role": "system", "content": f"""rewrite the tweets delimited by {delimiter} in the tone like Stewie """},
        {"role": "user", "content": f"{delimiter}{tweet}{delimiter} "}]

    print(openai_help(messages).strip(delimiter))

Ah, splendid! We dabble in the following delightful endeavors: 
• IoT Solutions, for those who fancy a connected world 
• Generative AI Integrations, because why not let machines do the thinking? 
• Mobile App Development, for the on-the-go genius 
• Web Development, because the internet simply cannot have enough websites 
• College Projects & MVP Prototypes, for the aspiring intellects 
Swift, dependable, and won't break the bank! 
Shall we create something magnificent together? 
Do take a gander: https://t.co/DXZqtPhy42 
#InnovateWithUs
RT @jradoff: Ah, the wonders of Artificial Intelligence Livestream! Join the illustrious Jon Radoff of Beamable as he welcomes the esteemed Emmanuel de Maistre of Scenario. Together, they shall delve into the enigmatic world of creators. Do tune in, won't you?
RT @jradoff: Ah, the wonders of Artificial Intelligence! Join the illustrious Jon Radoff of Beamable as he welcomes the esteemed Emmanuel de Maistre of Scenario. Together, they shall delve into 

### Inferring
- Use step-by-step instructions with delimiters to:
  1. Identify sentiments
  2. Identify emotions
  3. Extract mentioned people's names
  3. Identify whether a tweet supports Democratic, Republican, or unknown 
  4. Extract outputs into a structured JSON document. 
- Identify topics from Tweets. 


In [21]:
for tweet in tweet_data:
    messages = [
        {"role": "system", "content": f"""analyze the tweet delimited by {delimiter} in the following steps:
                                        step 1 {delimiter} identify the tweet sentiment in a single word, either positive, negative or neutral;
                                        step 2 {delimiter} identify the emotions expressed in the tweet with a single word;
                                        step 3 {delimiter} extract the mentioned peoples;
                                        step 4 {delimiter} detect whether the tweet support Democratic or Replublican, return the resunt in a single word;
                                        step 5 {delimiter} organize the result in a json document with the keys <sentiment>, <emontion>,<mentioned>, <support>
                                         Do not wrap the json codes in JSON markers and only return the json document"""},
        {"role": "user", "content": f"{delimiter}{tweet}{delimiter} "}]
    print(openai_help(messages))

{
  "sentiment": "positive",
  "emotion": "enthusiasm",
  "mentioned": [],
  "support": "neutral"
}
{
  "sentiment": "neutral",
  "emotion": "informative",
  "mentioned": ["Jon Radoff", "Emmanuel de Maistre"],
  "support": "neutral"
}
{
  "sentiment": "neutral",
  "emotion": "informative",
  "mentioned": ["Jon Radoff", "Emmanuel de Maistre"],
  "support": "neutral"
}
{
  "sentiment": "neutral",
  "emotion": "informative",
  "mentioned": [
    "DDNintelligence",
    "Gartner"
  ],
  "support": "neutral"
}
{
  "sentiment": "neutral",
  "emotion": "informative",
  "mentioned": ["Jon Radoff", "Emmanuel de Maistre"],
  "support": "neutral"
}
{
  "sentiment": "neutral",
  "emotion": "informative",
  "mentioned": ["mdancho84"],
  "support": "neutral"
}
{
  "sentiment": "positive",
  "emotion": "enthusiasm",
  "mentioned": [],
  "support": "neutral"
}
{
  "sentiment": "positive",
  "emotion": "excitement",
  "mentioned": ["@piyushgarg_dev", "@Hiteshdotcom"],
  "support": "neutral"
}
{
  "senti

In [22]:

messages = [
        {"role": "system", "content": f"""analyze the tweet delimited by {delimiter} to identify 10 topics, 
                                  Do not wrap the json codes in JSON markers """},
        {"role": "user", "content": f"{delimiter}{tweet_data}{delimiter} "}]
print(openai_help(messages))

{
  "topics": [
    "Generative AI in various industries",
    "Controversy and ethical concerns around generative AI",
    "Generative AI in art and creativity",
    "Generative AI in education and learning",
    "Generative AI in business and marketing",
    "Generative AI in technology and development",
    "Generative AI in healthcare and pharmaceuticals",
    "Generative AI in media and journalism",
    "Generative AI in collaboration and teamwork",
    "Generative AI in legal and regulatory discussions"
  ]
}


### Expanding with multiple prompts 
- Identify which party receives majority supports
- Provide contexts in the system message
- Create a chatbot to answer users’ inquiry  


In [23]:
analysis_result = []
from tqdm import tqdm
for tweet in tqdm(tweet_data):
    messages = [
        {"role": "system", "content": f"""analyze the tweet delimited by {delimiter} in the following steps:
                                        step 1 {delimiter} identify the tweet sentiment in a single word, either positive, negative or neutral;
                                        step 2 {delimiter} identify the emotions expressed in the tweet with a single word;
                                        step 3 {delimiter} extract the mentioned peoples;
                                        step 4 {delimiter} detect whether the tweet support Democratic or Replublican, return the resunt in a singple word;
                                        step 5 {delimiter} organize the result in a json document with the keys <sentiment>, <emontion>,<mentioned>, <support>
                                         Do not wrap the json codes in JSON markers and only return the json document"""},
        {"role": "user", "content": f"{delimiter}{tweet}{delimiter} "}]
    analysis_result.append(openai_help(messages))


100%|██████████| 93/93 [01:30<00:00,  1.03it/s]


In [24]:
print(analysis_result)

['{\n  "sentiment": "positive",\n  "emotion": "enthusiasm",\n  "mentioned": [],\n  "support": "neutral"\n}', '{\n  "sentiment": "neutral",\n  "emotion": "informative",\n  "mentioned": ["Jon Radoff", "Emmanuel de Maistre"],\n  "support": "neutral"\n}', '{\n  "sentiment": "neutral",\n  "emotion": "informative",\n  "mentioned": ["Jon Radoff", "Emmanuel de Maistre"],\n  "support": "neutral"\n}', '{\n  "sentiment": "neutral",\n  "emotion": "informative",\n  "mentioned": ["DDNintelligence", "Gartner"],\n  "support": "neutral"\n}', '{\n  "sentiment": "neutral",\n  "emotion": "informative",\n  "mentioned": ["Jon Radoff", "Emmanuel de Maistre"],\n  "support": "neutral"\n}', '{\n  "sentiment": "neutral",\n  "emotion": "informative",\n  "mentioned": ["mdancho84"],\n  "support": "neutral"\n}', '{\n  "sentiment": "positive",\n  "emotion": "enthusiasm",\n  "mentioned": [],\n  "support": "neutral"\n}', '{\n  "sentiment": "positive",\n  "emotion": "excitement",\n  "mentioned": [\n    "piyushgarg_dev",

In [25]:
messages = [
        {"role": "system", "content": f"""analyze the tweet analysis reuslt delimited by {delimiter} in the following steps:
                                        step 1 {delimiter} count the number of tweets that support Democratic and Republican;
                                        step 2 {delimiter} identify the common sentiments and emotoions to each mentioned people;
                                        step 3 {delimiter} organize the result in a json document with keys <Democratic count>, <Republican count>, <people name>
                                         Do not wrap the json codes in JSON markers and only return the json document"""},
        {"role": "user", "content": f"{delimiter}{analysis_result}{delimiter} "}]
analysis_summary = openai_help(messages)
print(analysis_summary)

{
  "Democratic count": 1,
  "Republican count": 1,
  "people name": {
    "Democratic": {
      "sentiments": ["neutral"],
      "emotions": ["amusement"]
    },
    "Republican": {
      "sentiments": ["negative"],
      "emotions": ["anger"]
    }
  }
}


## Create a chatbot

In [26]:
from openai import OpenAI

openai_api_key  = get_secret('openai')['api_key']
client = OpenAI(api_key=openai_api_key)
model = 'gpt-4o'
temperature = 0

chat_history = [

{"role": "system", "content": f"""you are a chabot answer user questions based on the tweets,
                                {delimiter}{tweet_data}{delimiter}, 
                                if user mentioned a people name in the {delimiter}{analysis_summary}{delimiter} people field,report the corresponding sentiment and emotion,
                            
                            """}
]

def chatbot(prompt):

    chat_history.append({"role": "user", "content": prompt})

    response = client.chat.completions.create(
        model=model,  # Use the model you prefer
        messages=chat_history
    )

    reply = response.choices[0].message.content

    chat_history.append({"role": "assistant", "content": reply})
    
    return reply

In [ ]:
while True:
    user_input = input("You: ")
    if user_input.lower() in ['exit', 'quit']:
        print("Chatbot: Goodbye!")
        break
    reply = chatbot(user_input)
    print(f"Chatbot: {reply}")

## Reference
- Isa Fulford and Andrew Ng. n.d.-a. *“Building Systems with the ChatGPT API.”* DeepLearning.AI. Accessed October 25, 2024. https://www.deeplearning.ai/short-courses/building-systems-with-chatgpt/.
- ———. n.d.-b. *“ChatGPT Prompt Engineering for Developers.”* DeepLearning.AI. Accessed October 25, 2024. https://www.deeplearning.ai/short-courses/chatgpt-prompt-engineering-for-developers/.
- OpenAI. n.d. *“OpenAI Documents.”* OpenAI. Accessed October 18, 2024. https://platform.openai.com.
